# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [2]:
Pkg.activate(".") # activate the parent environment
using BootstrapVirasoro
if Threads.nthreads() == 1
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`")
end

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`
[ Info: Precompiling BootstrapVirasoro [ca6c609c-7c6b-4ea6-be06-ee6e3ced7d6c] (cache misses: include_dependency fsize change (4), mismatched flags (16))


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [178]:
setprecision(BigFloat, 17, base=10)
Δmax = 20.

function LoopFields(c, model)
    model === :On && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=r, s=s) for r in 1//2:1:15 for s in -1+1//(2r):1//r:1 if (r*s)%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:15]
    )
    model === :PSUn && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
    model === :Potts && return vcat(
        [Field(c, r=r, s=s) for r in 2:15 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
end

LoopSpectrum(c, model, Δmax) = Spectrum(LoopFields(c, model), Δmax, interchiral=true);

function solve_bootstrap(c, indices, Δmax; signature = Dict(:s => 0, :t => 0, :u => 0), diags = Dict())
    fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(fields..., Δmax=Δmax)
    SOn = LoopSpectrum(:On, Δmax)

    Schan_On = ChannelSpectra(
        co, SOn,
        signature=signature
    )

    for (chan, VP) in diags
        add!(Schan_On, VP)
    end

    b = BootstrapSystem(Schan_On)
    compute_linear_system!(b)
    solve!(b)
    return b
end

solve_bootstrap (generic function with 2 methods)

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

We expect a three-dimensional space of solutions. We can distinguish between them by choosing an appropriate signature.

In [ ]:
using Random, BenchmarkTools
Random.seed!(1234)
c = CC(β=1/(big"0.8"+big"0.1"*im))
Δmax = 40.
setprecision(BigFloat, 35, base=10)
indices = ((1 // 2, 0), (1 // 2, 0), (1//2, 0), (1//2, 0))
ext_fields = [Field(c, r=r, s=s) for (r, s) in indices]

@time begin
    co = Correlation(ext_fields..., Δmax=Δmax)
    SOn = LoopSpectrum(c, :On, Δmax)
    S_even = Spectrum([V for V in SOn.fields if V.r % 1 == 0], Δmax, interchiral=true)

    chan_spectra = ChannelSpectra(
        co, SOn, (s=0, t=1, u=0);
        interchiral=true, Δmax=Δmax, parity=1,
        exclude=(u=[Field(c, r=2, s=0)],)
    )
    sys = BootstrapSystem(chan_spectra)
    evaluate_blocks!(sys)
    compute_linear_system!(sys)
    solve!(sys)
end

# on my machine the python code took 2min52sec  for Δmax = 20 and 13 working digits
#                                    8min10sec  for Δmax = 30 and 25 working digits
#                                    23min18sec for Δmax = 40 and 35 working digits
#               the julia  code took 4.4sec     for Δmax = 20 and 13 working digits
#                                    9.2sec     for Δmax = 30 and 25 working digits
#                                    24.5sec    for Δmax = 40 and 35 working digits

[ Info: system size: (112, 105)


 24.527959 seconds (212.42 M allocations: 8.725 GiB, 12.84% gc time, 1.56% compilation time)


In [479]:
sys.consts

Channel s
Field         | Structure constant           | Relative error             
-------------------------------------------------------------------
<1, 1>        |  1.00000e+00 + 0.00000e+00im | 0.00e+00
V_{(1, 0)}    |  1.03217e+00 - 2.43567e-01im | 2.56e-24
V_{(1, 1)}    | -4.19284e-01 + 2.91104e-01im | 2.08e-24
V_{(2, 0)}    | -8.17253e-03 + 2.99410e-03im | 6.22e-24
V_{(2, 1//2)} |  4.94044e-03 - 3.05524e-03im | 2.93e-24
V_{(2, 1)}    | -1.37830e-03 + 9.28059e-04im | 6.96e-24
V_{(3, 0)}    |  4.27790e-07 + 2.90061e-07im | 6.07e-18
V_{(3, 1//3)} | -4.80466e-07 - 2.79553e-07im | 5.06e-18
V_{(3, 2//3)} |  3.62837e-07 + 1.73428e-07im | 5.15e-18
V_{(3, 1)}    | -2.01447e-07 - 7.23303e-08im | 5.90e-18
V_{(4, 0)}    |  6.09057e-13 - 4.39886e-13im | 4.01e-09
V_{(4, 1//4)} | -6.58786e-13 + 4.16893e-13im | 3.66e-09
V_{(4, 1//2)} |  5.80623e-13 - 3.48893e-13im | 3.57e-09
V_{(4, 3//4)} | -3.93888e-13 + 3.13019e-13im | 3.65e-09
V_{(4, 1)}    |  2.42159e-13 - 2.60846e-13im | 3.51e-09
V_{(5, 

In [448]:
println(sort(sys.matrix.LHS[1, 1:12], by=real) ≈ sort(python_LHS[1, 1:12], by=real))
println(sort(sys.matrix.LHS[22, 25:end], by=real) ≈ sort(python_LHS[1, 25:end], by=real))
println(sort(sys.matrix.LHS[1, 13:24], by=real) ≈ sort(python_LHS[2, 13:24], by=real))

true
true
true


In [450]:
sys.positions

21-element Vector{Complex{BigFloat}}:
 0.230390691545437953369201977693592198193073 + 0.319620454526226782654418911988614127039909im
 0.187434661927532275615959633796592243015766 + 0.457698171280395249560513093456393107771873im
 0.241244657759684844045011686830548569560051 + 0.257702146966340306377674096438568085432053im
  0.48124985091393690517236336745554581284523 + 0.418218779013887764772761101994547061622143im
  0.29769994675616828772035660222172737121582 + 0.39936600875498962093246291260584257543087im
 0.102979202453146012619100702067953534424305 + 0.179750645639662198771446810496854595839977im
 0.482696381601963042662362113333074375987053 + 0.359142126308742315465849515021545812487602im
 0.144033704460453521045693037194723729044199 + 0.296553278318818391845468340761726722121239im
 0.326058143704515024374757103942101821303368 + 0.201524714483334449477425209806824568659067im
 0.149916782830760264566549722076160833239555 + 0.34394994626668079495956931168620940297842im
 0.36911715533

In [452]:
Block(co, :s, Field(c, r=3, s=0), interchiral=true, Δmax=Δmax)(sys.positions[1])

0.000692966061484028184605361625525102764697568 + 0.0048933220355258292971494257939457660476636im

In [376]:
python_LHS = 
[
  0.6977595399716+0.1347706022854im 0.4579828699472+0.05254518063945im 0.08696547797697+0.06735397734563im 0.05011742459289+0.05052067344305im -0.09090657935793-0.06078789071563im 0.0006929660614837+0.004893322035525im 0.0008173374773724+0.004206387763306im -0.001032326148919-0.005838787590878im -0.00138500534174-0.00714389057513im -0.03096574258168+0.1029993943722im -0.003954023793968+0.002782897174833im -0.00461173593778-0.001925476271289im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.462855448211+0.02305208170692im 1.507208279473-0.02142382467939im 2.783965206618-0.1943232825554im 11.00714656919-4.513237366999im -38.8822993719+88.4681427064im 35.05009639525-90.27683648657im 64.2954165085-124.4152457358im -104.66417919+179.6476324054im -0.4126141598639+0.1802651604455im 4.64717304237+6.514800780298im 100.9445787407-7.166114609482im -69.30606385446-107.2814702786im;
  0.6977595399716+0.1347706022854im 0.4579828699472+0.05254518063945im 0.08696547797697+0.06735397734563im 0.05011742459289+0.05052067344305im -0.09090657935793-0.06078789071563im 0.0006929660614837+0.004893322035525im 0.0008173374773724+0.004206387763306im -0.001032326148919-0.005838787590878im -0.00138500534174-0.00714389057513im -0.03096574258168+0.1029993943722im -0.003954023793968+0.002782897174833im -0.00461173593778-0.001925476271289im -1.270611316984+0.001777024461602im -1.965214002261-0.03110211003678im -2.257815324728+0.2271226297367im -2.834682930673+0.145225532935im -0.007768317806398+0.003290794109111im -6.266558928169+2.86634367397im -8.30003078824+3.24493571909im 3.836188248918-3.069860720877im 9.763449249309-4.184581902255im -1.346950520193+1.486275582566im -1.702510953534+6.431230711134im 4.830569517567+5.007485237708im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.7449878011775+0.1167590630155im 0.165955551341+0.01418748146125im 0.1549118421716+0.08341225602249im 0.02492770447511+0.02194159819408im -0.2447126589072-0.1211874892627im 0.006097268194391+0.01363208311426im 0.002565658796384+0.004004838159439im -0.007684083011845-0.01998757034654im -0.004545635136445-0.008550436525873im -0.06499410164924+0.1717877894087im -0.01165706992186+0.007636628226996im -0.008264911815885-0.007149533070711im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.299689195571+0.006589277073541im 1.273690557018-0.01387062730682im 1.772405175557+0.02771893461403im 5.616993748831-1.526557519583im -15.90248597047+16.45150819273im 14.93621810937-16.93384001286im 23.66657456735-22.31238628466im -35.96719460843+31.02982062102im -0.4564664880098+0.1005073110884im 2.074320503031+3.643377193817im 22.85439558064+6.638879668525im -6.733553970347-28.68588583392im;
  0.7449878011775+0.1167590630155im 0.165955551341+0.01418748146125im 0.1549118421716+0.08341225602249im 0.02492770447511+0.02194159819408im -0.2447126589072-0.1211874892627im 0.006097268194391+0.01363208311426im 0.002565658796384+0.004004838159439im -0.007684083011845-0.01998757034654im -0.004545635136445-0.008550436525873im -0.06499410164924+0.1717877894087im -0.01165706992186+0.007636628226996im -0.008264911815885-0.007149533070711im -1.231711145953-0.00264700698602im -1.439649989113-0.0003148196772247im -2.459569957593+0.4167683186651im -1.797463785808-0.04947571226353im 2.598949819961-0.4543608129958im -7.723611121977+4.92919743128im -8.360125952453+5.311164049464im 9.945498219009-6.530143861296im 16.83855288782-8.891643322009im -0.496832855227+2.161234125568im 0.3961741809921+9.679405337315im 10.63835831431+5.639209620548im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.6721566259471+0.1416851700924im 0.5711371637558+0.07362625523266im 0.06333721666657+0.05806249484059im 0.05365761496829+0.0570857939863im -0.03786322936011-0.02915888157068im -6.192881776827e-5+0.00273242888052im -5.303973319135e-5+0.002975632461295im -0.0002664641472613-0.002436084616892im -0.0001441176418666-0.00395868611432im -0.0164256489177+0.07722210369028im -0.002284024080433+0.001503940904174im -0.00266511704996-0.001097351053404im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.542370885443+0.03332204050929im 1.65154752705-0.02614799204299im 3.459908822237-0.3851040772082im 14.8639221876-7.148961761456im -46.32988716316+187.3222716222im 39.58205768301-191.935568371im 86.09756341353-267.7889136781im -152.2758677554+394.9018585148im -0.3942071648259+0.2207360356249im 6.737065507236+8.480303335779im 197.1264918531-49.27525269952im -175.1337098893-191.0499312035im;
  0.6721566259471+0.1416851700924im 0.5711371637558+0.07362625523266im 0.06333721666657+0.05806249484059im 0.05365761496829+0.0570857939863im -0.03786322936011-0.02915888157068im -6.192881776827e-5+0.00273242888052im -5.303973319135e-5+0.002975632461295im -0.0002664641472613-0.002436084616892im -0.0001441176418666-0.00395868611432im -0.0164256489177+0.07722210369028im -0.002284024080433+0.001503940904174im -0.00266511704996-0.001097351053404im -1.298667586577+0.008786701253329im -2.222726710859-0.04743159627259im -2.282645196063+0.2280410728415im -3.514405409751+0.3311540798892im -1.315580958983+0.1683639763048im -6.083035190709+2.377169042639im -8.999342123481+3.018906869633im 0.05637764502254-1.211592176911im 5.365771884552-2.200623356769im -1.776669042091+1.26770699585im -2.816062103536+5.411945883331im 1.949956113417+4.743532762191im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.9406311984124+0.08135908782751im 0.9092066052974+0.04764793162412im 0.5420901038506+0.124524409326im 0.4064195593854+0.1575502292394im -0.4815189390027-0.09194657252615im 0.2236390816434+0.09255094520222im 0.2361365955195+0.09499689947148im -0.2668586347756-0.09363513190635im -0.401991144818-0.1737953399148im 0.1056308220136+0.5384998464232im 0.0398078518303+0.2471811238532im -0.2203503344356+0.1903559329715im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.3373903007-0.01178343313275im 0.08832669779828-0.001190522955334im 0.1112903847652+0.01383381523067im 11.47797823597-4.028677011578im -26.32514357229+29.384511314im 1.81849599783-3.206838636422im 42.96976831273-39.6049418983im -5.224927744338+6.176401784912im -0.4286936640493+0.1283359288471im 2.887908442966+5.921361761555im 28.38609386389+24.1977931658im 17.90085759116-22.1555714949im;
  0.9406311984124+0.08135908782751im 0.9092066052974+0.04764793162412im 0.5420901038506+0.124524409326im 0.4064195593854+0.1575502292394im -0.4815189390027-0.09194657252615im 0.2236390816434+0.09255094520222im 0.2361365955195+0.09499689947148im -0.2668586347756-0.09363513190635im -0.401991144818-0.1737953399148im 0.1056308220136+0.5384998464232im 0.0398078518303+0.2471811238532im -0.2203503344356+0.1903559329715im -0.9742217373582-0.07335049902705im -0.9975343366626-0.04645681654019im -0.6552079196875-0.1200549608667im -0.5177436195094-0.1713325219664im 0.5433893116407+0.07965220670887im -0.3594912271648-0.1029402107968im -0.3882332306623-0.1108205555606im 0.4190691198951+0.09141310992906im 0.6543188176246+0.2033696006402im -0.3515456065321+0.459873296095im -0.2782135661961+0.2632237145941im 0.1127731563485+0.4310339806879im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.7861866502296+0.1150093632808im 0.5066709902979+0.04283253435346im 0.1952870838269+0.09714567399603im 0.09923940259773+0.07281548945345im -0.2352939517888-0.1038008505542im 0.0138395025873+0.0212385186127im 0.01256810462849+0.016670978954im -0.01799045674598-0.02735340340371im -0.02202574975392-0.03174167026513im -0.0334140719626+0.218969869115im -0.01303650303726+0.02110924699427im -0.02597471675404-0.0001840538533577im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.362690171864+0.001251355742255im 0.9701060040999-0.01269013018153im 1.398561272359+0.05745437368745im 9.910189320031-3.456173912384im -26.85913822434+37.11655062318im 18.67905258972-32.90373997813im 43.57598036712-51.19154235478im -52.17888850086+63.59650355659im -0.4299493134701+0.1358637306485im 3.172481392087+5.537480584943im 44.21928033536+9.771367434013im -14.98078616837-49.28475117614im;
  0.7861866502296+0.1150093632808im 0.5066709902979+0.04283253435346im 0.1952870838269+0.09714567399603im 0.09923940259773+0.07281548945345im -0.2352939517888-0.1038008505542im 0.0138395025873+0.0212385186127im 0.01256810462849+0.016670978954im -0.01799045674598-0.02735340340371im -0.02202574975392-0.03174167026513im -0.0334140719626+0.218969869115im -0.01303650303726+0.02110924699427im -0.02597471675404-0.0001840538533577im -1.155389867438-0.02731752477724im -1.476786926671-0.03013626744203im -1.57481901026+0.04431609550871im -1.498113832271-0.1297540685034im 0.9212209146361-0.05254691139739im -2.890765998508+0.7523480468701im -3.396471664932+0.770554433694im 3.007425269564-1.106768179172im 5.50845426674-1.149788141547im -0.7199465239465+1.149680410128im -0.9790845012478+2.865019495287im 2.347947630069+2.663784673877im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.4946398712031+0.1646603144788im 0.2594934936984+0.0512262779008im 0.006403039789217+0.01583535801327im 0.003374486565455+0.01054729564103im -0.006802071313892-0.01223333299197im -4.992399236405e-5+2.831849639555e-5im -3.897541881197e-5+2.468526983461e-5im 5.281157397944e-5-3.432574864298e-5im 5.459646613203e-5-3.992100159985e-5im -0.0110774844266+0.01110819131723im -4.498016593344e-5-3.06548831019e-5im 1.538019765517e-6-5.033158906807e-5im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.763635838075+0.114430833839im 3.031780996414-0.05206527311173im 10.43770057519-3.820471690962im 5.519799065208-2.869970594814im -41.20540905353+373.5801825839im 70.54524094382-431.6224214044im 30.51072185764-462.534508994im -182.5833089985+782.0691973118im -0.3929776744427+0.2999109819852im 9.039378653548+5.851906641767im 372.1948884376-184.7006393701im -431.9176540996-300.5635804595im;
  0.4946398712031+0.1646603144788im 0.2594934936984+0.0512262779008im 0.006403039789217+0.01583535801327im 0.003374486565455+0.01054729564103im -0.006802071313892-0.01223333299197im -4.992399236405e-5+2.831849639555e-5im -3.897541881197e-5+2.468526983461e-5im 5.281157397944e-5-3.432574864298e-5im 5.459646613203e-5-3.992100159985e-5im -0.0110774844266+0.01110819131723im -4.498016593344e-5-3.06548831019e-5im 1.538019765517e-6-5.033158906807e-5im -1.645856430406+0.1095214709696im -3.291339720423+0.000936093687863im -5.862062495313+2.062476636109im -10.44179882383+3.815925575834im -6.280580610927+2.200303281892im -39.00751780987+49.49376691769im -64.0173883469+75.37118898468im -21.12105187613+14.3159169149im 14.63820236109-19.50218346105im -4.931410075237+4.132665483693im -4.492134429302+58.98837227163im 24.2399818338+38.33004582196im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.9354098465635+0.0851589475868im 1.050953429368+0.06320106462665im 0.4879596613729+0.1332228587911im 0.4700268078101+0.1751458266969im -0.240434012847-0.05308235838129im 0.1710050629674+0.08659016141658im 0.1983198525764+0.1060626310177im -0.1687411139894-0.05762939572565im -0.2852472420738-0.1452816400479im 0.1346406867157+0.4506440620862im 0.03171146970519+0.1977069148819im -0.1695262456279+0.1512624571508im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.392969206411-0.00790963182442im 0.09142111507595-0.001346763759236im 0.1263583554585+0.01263635975283im 14.64009163691-5.973543322841im -38.19063298042+55.47134683956im 2.366652902518-5.81462786139im 64.25965977396-76.32383242733im -7.505286377671+11.55951050389im -0.4115134643604+0.1574697019567im 4.135090747907+7.417783966618im 53.30052519095+34.83063325687im 25.56379081327-41.36011514658im;
  0.9354098465635+0.0851589475868im 1.050953429368+0.06320106462665im 0.4879596613729+0.1332228587911im 0.4700268078101+0.1751458266969im -0.240434012847-0.05308235838129im 0.1710050629674+0.08659016141658im 0.1983198525764+0.1060626310177im -0.1687411139894-0.05762939572565im -0.2852472420738-0.1452816400479im 0.1346406867157+0.4506440620862im 0.03171146970519+0.1977069148819im -0.1695262456279+0.1512624571508im -0.9701187556251-0.07682551421575im -1.142376201652-0.06185308761235im -0.5937885737112-0.1329502811187im -0.5964329574961-0.187687441937im 0.2532038493724+0.04431818163137im -0.2796014868075-0.1038938279871im -0.3304938194782-0.1336644672775im 0.2624339790632+0.05378767144137im 0.4631508426393+0.1754194004652im -0.398821209348+0.3380352704946im -0.2430701361531+0.1667667169069im 0.02883059091999+0.3210131017175im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.6225792784114+0.1461872769673im 0.258913096719+0.03561396687708im 0.04149793536767+0.04409360914469im 0.01491347576393+0.02131496860116im -0.05380048286329-0.04882279089942im -0.0002896832179026+0.00114665167863im -0.0001090037070935+0.0007028122137857im 0.0003915792141412-0.001510116225893im 0.0002124346017213-0.001272893858052im -0.03135859421626+0.0508435788917im -0.001110241472131+8.908654259006e-5im -0.0005286683162677-0.0008362720899509im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.510309055447+0.04770829724899im 2.041640849223-0.02801930975181im 4.497869344044-0.7578234593861im 7.319151425089-2.954113604804im -39.71398905135+95.24198771827im 45.13024088157-104.0935996348im 56.39947044653-129.4196510704im -114.6366284492+199.3033748493im -0.4164658641113+0.1936038165345im 4.714958496563+5.32726113543im 111.2426474371-15.24405879589im -85.63821519148-116.7143830848im;
  0.6225792784114+0.1461872769673im 0.258913096719+0.03561396687708im 0.04149793536767+0.04409360914469im 0.01491347576393+0.02131496860116im -0.05380048286329-0.04882279089942im -0.0002896832179026+0.00114665167863im -0.0001090037070935+0.0007028122137857im 0.0003915792141412-0.001510116225893im 0.0002124346017213-0.001272893858052im -0.03135859421626+0.0508435788917im -0.001110241472131+8.908654259006e-5im -0.0005286683162677-0.0008362720899509im -1.399604543329+0.03782045885092im -2.3005760669-0.007574637470798im -3.522449251941+0.7489854941956im -4.513331469556+0.7379497711086im -0.1403434281832+0.03958026707783im -15.92390743143+13.65046278812im -21.53763990048+16.2997810774im 9.321796418619-12.75318040031im 26.15175611506-20.84580457853im -1.918763532341+2.557850591847im 0.5580979466766+20.29849504208im 18.80237739573+10.67250353886im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.7326805111563+0.1325715437709im 0.8902949494496+0.1056029970905im 0.1019440286841+0.0767475418787im 0.140848916929+0.1069342131682im 0.03239218990758+0.0205263010028im 0.001290439988554+0.006738753323155im 0.001613621808915+0.009775012942763im -0.0009141448241053+0.0002130819810772im -0.001333622990143-0.004960521576675im 0.02663409247594+0.1055577591647im -0.003426983472155+0.005830108832756im -0.005943747118458+0.001181334850185im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.575354908562+0.02396685398133im 1.220392375482-0.02154075077423im 2.442023433539-0.1353546932637im 24.37466192353-13.90794550466im -44.11443149161+344.5849844348im 7.929532052531-303.9441948334im 109.6243596596-501.3957312641im -140.5857400677+650.7767288939im -0.3725829222779+0.2494372655782im 10.27697246087+12.4042940003im 331.9240002405-102.776342186im -296.8866238449-283.205161241im;
  0.7326805111563+0.1325715437709im 0.8902949494496+0.1056029970905im 0.1019440286841+0.0767475418787im 0.140848916929+0.1069342131682im 0.03239218990758+0.0205263010028im 0.001290439988554+0.006738753323155im 0.001613621808915+0.009775012942763im -0.0009141448241053+0.0002130819810772im -0.001333622990143-0.004960521576675im 0.02663409247594+0.1055577591647im -0.003426983472155+0.005830108832756im -0.005943747118458+0.001181334850185im -1.194651674156-0.02023893616793im -2.110742646984-0.08397928241493im -1.44503529918-0.007112084555512im -2.583470208474+0.03353832583099im -1.547677897358+0.01474609057095im -2.036337886773+0.1216366289909im -3.385675268341+0.1362367137831im -1.570138005446-0.02616782889494im -0.07121522363993+0.007444490968744im -1.390102575218+0.5155709821624im -1.658471880774+1.027125722511im -0.8239997527481+1.272323061252im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.6590545266777+0.138203223378im 0.220062945012+0.02661883860491im 0.06391289404879+0.05538899454595im 0.01734049172023+0.02127192486246im -0.09037395143616-0.06866572946077im -0.0001000917484371+0.002619252856457im 9.866289371918e-5+0.001260751869629im 0.0002378258238952-0.003602664505016im -9.710505957343e-5-0.002417620383728im -0.04144631503138+0.07447622084158im -0.002393328665144+0.0005319096691165im -0.001215207453957-0.001659800182098im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.438263399167+0.0333843113146im 1.801347629349-0.02280819273468im 3.448159212273-0.3886261137983im 6.622610001003-2.377095366734im -30.98560140533+54.83814334486im 33.51193181197-59.13403196076im 44.50133086706-74.73462045585im -82.26502479631+112.0369883562im -0.4288466755055+0.1613580856194im 3.650307900482+4.689529325236im 67.92372689622-0.6264209806668im -42.52252438966-76.02025239459im;
  0.6590545266777+0.138203223378im 0.220062945012+0.02661883860491im 0.06391289404879+0.05538899454595im 0.01734049172023+0.02127192486246im -0.09037395143616-0.06866572946077im -0.0001000917484371+0.002619252856457im 9.866289371918e-5+0.001260751869629im 0.0002378258238952-0.003602664505016im -9.710505957343e-5-0.002417620383728im -0.04144631503138+0.07447622084158im -0.002393328665144+0.0005319096691165im -0.001215207453957-0.001659800182098im -1.345543508344+0.02378789565954im -2.021423576476-0.003800721438774im -3.180582734505+0.6280929439054im -3.465865402344+0.3681165150768im 1.112757197513-0.2471098602143im -13.16970121679+10.79847419413im -16.60068135217+12.24722779144im 11.7958293958-12.48005224751im 25.75518475613-18.58610116741im -1.336287636931+2.460553498302im 1.233983373058+17.00816441949im 17.8011818793+8.74856736158im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.8430039707834+0.1036359473525im 0.6694049589218+0.04836002795824im 0.2915631785383+0.1136257081214im 0.1833496532802+0.1069345239429im -0.3055279088428-0.1039838546711im 0.04390244139354+0.04165745588838im 0.04313472842356+0.03784596164299im -0.0553547611015-0.04946982554564im -0.07470156570571-0.06958689763933im -0.001649201395991+0.3133258666717im -0.01537355844069+0.05888706313565im -0.06469686869829+0.02009957072229im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.353352127887-0.006015820453764im 0.6283223804331-0.008439640203017im 0.8466949314397+0.07230339223236im 11.10038464337-3.944680244958im -27.52067347618+34.9966419547im 12.88922806984-23.46803803194im 45.08103417949-47.85460123644im -36.98436660409+45.44362422332im -0.4275182861777+0.1342737831453im 3.203327378183+5.911481941114im 39.27486689903+15.24718610925im -3.723572137959-40.55250957952im;
  0.8430039707834+0.1036359473525im 0.6694049589218+0.04836002795824im 0.2915631785383+0.1136257081214im 0.1833496532802+0.1069345239429im -0.3055279088428-0.1039838546711im 0.04390244139354+0.04165745588838im 0.04313472842356+0.03784596164299im -0.0553547611015-0.04946982554564im -0.07470156570571-0.06958689763933im -0.001649201395991+0.3133258666717im -0.01537355844069+0.05888706313565im -0.06469686869829+0.02009957072229im -1.081982914706-0.04671061000116im -1.297734650518-0.03991596506185im -1.124221093661-0.06148355889831im -1.03027730213-0.1788626384495im 0.7088011757656+0.01557993614233im -1.324247815886+0.06024536070357im -1.52474636644+0.03291172888919im 1.408025310737-0.1996715236721im 2.457636026418-0.01866545430693im -0.5631388985662+0.7896734968038im -0.6794361842439+1.159402147252im 0.7868492196799+1.351862144937im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.7590770268398+0.1266953009387im 0.8546267091655+0.09225071113464im 0.1315779594809+0.08727849262295im 0.155281366146+0.1105077251309im -0.006240793068422-0.003498711486815im 0.003873149973128+0.0109028171578im 0.004686799121472+0.01442327491355im -0.003420261750629-0.004107587276736im -0.005255981851727-0.01258078460343im 0.02081033333596+0.1363904063022im -0.005475895109511+0.01032990096544im -0.01159193040353+0.001770072162335im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.523661167172+0.01524203665567im 1.06845188031-0.01786958931573im 1.960660649647-0.04421220542531im 20.4007381867-10.54508990981im -54.00471249793+207.1226298527im 24.24953244645-174.3220664936im 106.3941702754-297.6875119571im -122.580394397+365.3207853684im -0.3840945939235+0.2218570690758im 7.949929571905+10.50589642233im 208.6754105339-31.37043573119im -147.0123262925-190.1420955685im;
  0.7590770268398+0.1266953009387im 0.8546267091655+0.09225071113464im 0.1315779594809+0.08727849262295im 0.155281366146+0.1105077251309im -0.006240793068422-0.003498711486815im 0.003873149973128+0.0109028171578im 0.004686799121472+0.01442327491355im -0.003420261750629-0.004107587276736im -0.005255981851727-0.01258078460343im 0.02081033333596+0.1363904063022im -0.005475895109511+0.01032990096544im -0.01159193040353+0.001770072162335im -1.163875713329-0.02867356180449im -1.923115570788-0.07433689323948im -1.329288703819-0.04336030408339im -2.11660557222-0.06339126468943im -0.9114425617146-0.001639506413056im -1.759198729188+0.06197843476573im -2.667515898811-0.00886177245772im -0.3285076117297-0.2363943488744im 1.062882632914-0.0732541245927im -1.153628209055+0.5605633011593im -1.273113044125+1.04225285088im -0.1675732148657+1.282109021266im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.7999417889244+0.1140757373973im 0.6334974061474+0.05386319617233im 0.2076287929921+0.1021591468814im 0.1368369767141+0.09281159024009im -0.2019898621439-0.08636210753483im 0.0170262377437+0.02394623213125im 0.01714481831211+0.02248783352173im -0.02134388689555-0.02735605312119im -0.02878842200501-0.03878662117647im -0.01482391383435+0.2274165306327im -0.01246876301235+0.02672643370501im -0.03220701539739+0.004100364405395im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.390257033783+0.0007251002977606im 0.8703120660892-0.01211008328342im 1.286950705385+0.05698346219901im 11.91028875952-4.544287536186im -32.81730421803+51.2818739769im 19.5999751749-41.77415318657im 54.57383338642-71.18596753045im -58.26126225589+82.20516823022im -0.4197119546434+0.1512977859054im 3.819935091253+6.43307838792im 58.06804066309+11.39474039831im -19.68299945156-61.19766517028im;
  0.7999417889244+0.1140757373973im 0.6334974061474+0.05386319617233im 0.2076287929921+0.1021591468814im 0.1368369767141+0.09281159024009im -0.2019898621439-0.08636210753483im 0.0170262377437+0.02394623213125im 0.01714481831211+0.02248783352173im -0.02134388689555-0.02735605312119im -0.02878842200501-0.03878662117647im -0.01482391383435+0.2274165306327im -0.01246876301235+0.02672643370501im -0.03220701539739+0.004100364405395im -1.130752559259-0.03511581568134im -1.503821593278-0.04174472069757im -1.348809023667-0.02498094608544im -1.42414861492-0.1491240555119im 0.5001032952411-0.008221147141567im -2.008477983911+0.2855014959586im -2.449139341728+0.2463934192765im 1.807318631198-0.5411533608026im 3.518267868796-0.3862373763247im -0.7464268266537+0.8919511939848im -0.9291538326783+1.785560288519im 1.218033880676+1.900538184777im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.7127118531675+0.1369302213492im 0.981491647787+0.1269194891238im 0.08225115251433+0.06774082849654im 0.1438328716359+0.1109126983608im 0.08011741093541+0.05667564289457im 0.0002572646473744+0.004352393085861im 0.0004187115138665+0.007428911961777im 0.0002771393761506+0.00425700902637im 0.0001761118796706+0.001209859945124im 0.04520570925209+0.08094175840228im -0.00159083877003+0.003762585580827im -0.002246295975792+0.002076253850903im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.646715837181+0.03647326257926im 1.329430969402-0.02525836808909im 2.918655372816-0.2462201473354im 33.74359803636-22.61698332706im 50.32178983757+751.5165394761im -108.9285468699-668.7005047506im 25.40229569365-1114.032572641im -43.34623459239+1484.482005284im -0.353920050018+0.2916662872336im 15.76134872612+16.57349773389im 664.1182027497-366.351745393im -771.55015562-483.7920451283im;
  0.7127118531675+0.1369302213492im 0.981491647787+0.1269194891238im 0.08225115251433+0.06774082849654im 0.1438328716359+0.1109126983608im 0.08011741093541+0.05667564289457im 0.0002572646473744+0.004352393085861im 0.0004187115138665+0.007428911961777im 0.0002771393761506+0.00425700902637im 0.0001761118796706+0.001209859945124im 0.04520570925209+0.08094175840228im -0.00159083877003+0.003762585580827im -0.002246295975792+0.002076253850903im -1.2153776191-0.01388802987544im -2.311010487295-0.1014567739366im -1.493593130603+0.03346034076113im -3.061731563903+0.1467739733816im -2.369757340168+0.01128585895674im -2.073288714188+0.1667114564634im -3.933902570397+0.3435478947496im -3.58132677303+0.4273093307962im -2.265996065176+0.2136476183184im -1.614923143723+0.3705741278582im -2.015195904921+0.7156957123294im -1.906946108224+0.9373675685015im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.7803408880536+0.1236494910865im 1.136408813146+0.1271989648394im 0.1438888193558+0.09041408430541im 0.2567759348695+0.150286613625im 0.1501991497197+0.08236629982073im 0.004457419148597+0.01235985194828im 0.007648418086462+0.02138398871094im 0.004865263235537+0.01302437957134im 0.001951377129073+0.004361321146674im 0.09213342921879+0.1180582911417im -0.001168087252036+0.01224944755068im -0.004311496511676+0.008222620720892im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.612472325282+0.0240599749372im 0.9035804484627-0.01709626633905im 1.790101530919-0.04421791645154im 34.72696030889-23.09006558738im -10.29818431472+616.3697607014im -57.86677706413-434.1555247519im 101.5917396622-906.813788848im -59.67509760722+959.1838438081im -0.3537873624153+0.2789296730912im 15.37824852805+16.67883671454im 559.450200465-193.365712466im -457.8586379654-421.1882334429im;
  0.7803408880536+0.1236494910865im 1.136408813146+0.1271989648394im 0.1438888193558+0.09041408430541im 0.2567759348695+0.150286613625im 0.1501991497197+0.08236629982073im 0.004457419148597+0.01235985194828im 0.007648418086462+0.02138398871094im 0.004865263235537+0.01302437957134im 0.001951377129073+0.004361321146674im 0.09213342921879+0.1180582911417im -0.001168087252036+0.01224944755068im -0.004311496511676+0.008222620720892im -1.121972705255-0.0398816049315im -2.040050360435-0.1099723825047im -1.025894892478-0.07816164452439im -2.046627555675-0.09864127157507im -1.508153900075-0.1182397891455im -0.8741235320383-0.1229018131484im -1.627685865731-0.2208773460847im -1.388706440999-0.143043876294im -0.8145905169012-0.09210148365049im -1.102624091356+0.1767523328219im -0.884041210876+0.1388631139415im -0.8114256147104+0.2556859300104im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.8705356310814+0.1032058667361im 1.172642489558+0.09930947865994im 0.2889222700671+0.1236292646099im 0.4091261607331+0.1789043648473im 0.1055160714475+0.03767200609219im 0.03987737063236+0.04299979769029im 0.0555985412511+0.06416601493888im -0.007575010418155+0.0047403856532im -0.03398057412021-0.03301172596966im 0.1290157155081+0.2365704034907im 0.001087831054584+0.05847195160849im -0.04008081166947+0.03555873927574im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.508475333695+0.004025542130475im 0.4033058979416-0.006942639135301im 0.67003910178+0.03030488342097im 23.76145359993-12.78099917044im -65.54676978951+200.818773724im 8.926202937204-78.14267489535im 125.6447957043-286.5201663262im -52.22998488686+164.584739453im -0.3785289738383+0.2207357017041im 8.613449804048+11.65607928056im 194.7615223448+28.14312053416im -27.18738922673-159.8555856003im;
  0.8705356310814+0.1032058667361im 1.172642489558+0.09930947865994im 0.2889222700671+0.1236292646099im 0.4091261607331+0.1789043648473im 0.1055160714475+0.03767200609219im 0.03987737063236+0.04299979769029im 0.0555985412511+0.06416601493888im -0.007575010418155+0.0047403856532im -0.03398057412021-0.03301172596966im 0.1290157155081+0.2365704034907im 0.001087831054584+0.05847195160849im -0.04008081166947+0.03555873927574im -1.023242547119-0.06605379599702im -1.575964678303-0.09234675409092im -0.6907937501295-0.1353486463132im -1.079443320485-0.2079028328058im -0.436698868909-0.06669902804617im -0.3681133794086-0.1323828635217im -0.5526663639978-0.2202649543494im -0.06256605343524-0.07725272259812im 0.2095929153897+0.06507954785256im -0.6373180778935+0.205909748483im -0.3496281018284+0.1077793201351im -0.1443499402067+0.2454566300853im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.8281065751766+0.1130504388283im 1.129929830414+0.1086286034768im 0.2120363230637+0.1094918412343im 0.3211390015646+0.1635500400987im 0.116777082639+0.05112760042248im 0.01619097520927+0.02540761328766im 0.02360079344303+0.03927442173583im 0.001087717659612+0.008270697471415im -0.009080949330943-0.01239275776244im 0.0999510884482+0.1778315618629im -0.002736134672522+0.02930134847646im -0.01908800119551+0.01602878599172im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.543128627112+0.0103419363414im 0.6366382627853-0.01131961741682im 1.128716833262+0.02042409897763im 26.57628814456-15.27511067167im -63.6635099277+289.2197502791im 6.21532369337-163.3419907453im 134.9581231012-417.1285764419im -82.31568015027+349.1869861424im -0.3707257717573+0.2392816869874im 10.36259493019+12.99969948521im 277.8733176399-14.31520177495im -121.639166325-228.8176185779im;
  0.8281065751766+0.1130504388283im 1.129929830414+0.1086286034768im 0.2120363230637+0.1094918412343im 0.3211390015646+0.1635500400987im 0.116777082639+0.05112760042248im 0.01619097520927+0.02540761328766im 0.02360079344303+0.03927442173583im 0.001087717659612+0.008270697471415im -0.009080949330943-0.01239275776244im 0.0999510884482+0.1778315618629im -0.002736134672522+0.02930134847646im -0.01908800119551+0.01602878599172im -1.069040493921-0.05428538512759im -1.766598728657-0.09726483960063im -0.8420594545334-0.1193244853785im -1.450228872129-0.181224146439im -0.7911365040781-0.08892617455193im -0.575785820182-0.1461782386409im -0.9340721253618-0.2585937946868im -0.3654635103458-0.1370274037863im 0.05157740652404+0.01052799146588im -0.8209246865984+0.2196767018027im -0.5424788515238+0.1544177409107im -0.3067429133362+0.3038469541156im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.89952421073+0.0925161066585im 0.8923161005217+0.05709035113322im 0.4058593000366+0.1270245508691im 0.3324693332629+0.1494687777812im -0.3018667810299-0.07915066839704im 0.1069890761753+0.06822782053309im 0.1164522388647+0.07435991008555im -0.1212167312799-0.06352432364933im -0.1858980878519-0.119338582533im 0.06759501365996+0.403494948542im 0.001422946717209+0.131259635506im -0.1264347420344+0.08156297701015im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.37185052736-0.00849286214955im 0.2962654135384-0.004198384822326im 0.3989001349789+0.04085831591323im 13.07985705031-5.014754810543im -32.81401886479+43.60637400706im 6.99578588313-14.83916621635im 54.49893926044-59.66032435332im -21.13890434458+29.13584632629im -0.4187669113673+0.145777107402im 3.664059964176+6.726505110586im 44.41290383349+24.88888924662im 10.56258808757-39.07934112763im;
  0.89952421073+0.0925161066585im 0.8923161005217+0.05709035113322im 0.4058593000366+0.1270245508691im 0.3324693332629+0.1494687777812im -0.3018667810299-0.07915066839704im 0.1069890761753+0.06822782053309im 0.1164522388647+0.07435991008555im -0.1212167312799-0.06352432364933im -0.1858980878519-0.119338582533im 0.06759501365996+0.403494948542im 0.001422946717209+0.131259635506im -0.1264347420344+0.08156297701015im -1.012086495242-0.06557839845859im -1.188585961009-0.05288901355637im -0.7665289010614-0.1188548473489im -0.731504838382-0.1900867764501im 0.4098146202724+0.04647800429675im -0.5271699948354-0.1045314158266im -0.612732391929-0.1376533162409im 0.530213075718+0.03890548930249im 0.9213946528896+0.2017506507992im -0.4576171936641+0.4847465220353im -0.381015325258+0.3795803239622im 0.1638384390691+0.5844451794233im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.7941975993838+0.1096379645449im 0.3696319411868+0.02813371045368im 0.2221182253404+0.09811454337251im 0.07524834727085+0.05501747907071im -0.3216259313061-0.1299524933114im 0.01921237211051+0.02605089904959im 0.01377176164842+0.01481869921661im -0.02541092128947-0.03700131389997im -0.02514663812111-0.03133643073432im -0.05011203052499+0.2496001058075im -0.01753423267627+0.02527219829349im -0.02872495467519-0.003414284400066im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.306708145908-0.002368349799522im 0.9499852426579-0.01123687485391im 1.247575443184+0.08036310329265im 7.632988738184-2.262495328454im -18.49301728156+19.81137969188im 13.66256793714-18.09698561211im 29.07499646806-26.87873548468im -35.53972485815+33.82535786711im -0.4472328629062+0.1077645928023im 2.299311025879+4.408269203771im 25.38049663217+8.848840009868im -5.481133937469-30.23080375732im;
  0.7941975993838+0.1096379645449im 0.3696319411868+0.02813371045368im 0.2221182253404+0.09811454337251im 0.07524834727085+0.05501747907071im -0.3216259313061-0.1299524933114im 0.01921237211051+0.02605089904959im 0.01377176164842+0.01481869921661im -0.02541092128947-0.03700131389997im -0.02514663812111-0.03133643073432im -0.05011203052499+0.2496001058075im -0.01753423267627+0.02527219829349im -0.02872495467519-0.003414284400066im -1.156493297952-0.0241747132535im -1.319622869879-0.01689395569106im -1.725133667769+0.1212068172292im -1.32301794427-0.1351144490247im 1.642553621214-0.140450859504im -3.555628373375+1.226776089945im -3.87352177194+1.317518648623im 4.379299913699-1.673094423809im 7.285057310428-1.99709741248im -0.5382406103382+1.4219201825im -0.7828314902278+3.854443089408im 3.605852204838+3.167259767088im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.8638211699492+0.1038395981963im 1.044375773826+0.08500538608471im 0.2893124344173+0.1227798316976im 0.3417140945075+0.1620899383525im -0.01998786297867-0.007093566826832im 0.04209059516036+0.04290696093972im 0.05286181868495+0.05769979896908im -0.02895184267242-0.01527488771183im -0.05621910502757-0.05371020242682im 0.08722384199972+0.2627666097719im -0.004312494810623+0.0614902820719im -0.0546174992623+0.03328981407633im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.46615397901+0.0002901763177448im 0.4574009575257-0.007415719182343im 0.7198238256593+0.04057033026243im 19.34734289338-9.343514005666im -55.06464386851+124.2139465331im 12.85267716274-56.68079141893im 98.79404561935-175.2454481543im -51.31447026495+116.787781085im -0.3915725182894+0.1961438392629im 6.537481612796+9.685589493118im 123.8131753642+27.04812774473im -15.38270452991-106.9928807826im;
  0.8638211699492+0.1038395981963im 1.044375773826+0.08500538608471im 0.2893124344173+0.1227798316976im 0.3417140945075+0.1620899383525im -0.01998786297867-0.007093566826832im 0.04209059516036+0.04290696093972im 0.05286181868495+0.05769979896908im -0.02895184267242-0.01527488771183im -0.05621910502757-0.05371020242682im 0.08722384199972+0.2627666097719im -0.004312494810623+0.0614902820719im -0.0546174992623+0.03328981407633im -1.037185976695-0.06189294770597im -1.501790226779-0.07757616826166im -0.7754303142716-0.1299678437174im -1.061842087633-0.2017222849139im -0.2005650316229-0.02387920498122im -0.5106431400816-0.1297197259087im -0.7017855661336-0.2123663964454im 0.1641893478884-0.05271515070316im 0.558448406545+0.1280028504im -0.6328828159893+0.3253950205642im -0.4269555266241+0.2515055601817im -0.03980779036769+0.4405947615558im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.9381411884153+0.08627890730181im 1.208990212119+0.08001085583895im 0.4626445513386+0.1377396154157im 0.5640732960706+0.1930940899081im -0.006104621224184-0.001583212382786im 0.1444312379959+0.08462915186578im 0.1855783265566+0.1188869215868im -0.08773359870338-0.01829888136588im -0.1906494204387-0.1061186713289im 0.1850499768677+0.3818468039884im 0.03425237452596+0.1692935172775im -0.1238640130947+0.1308309654496im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.448857399718-0.003458423203189im 0.05416386442852-0.0008671596551546im 0.08156137909702+0.00636586867779im 18.76478374578-8.825451772004im -53.06126349167+103.9945306668im 1.561091813183-6.037840999383im 93.34042649679-145.6342123317im -5.896442480899+12.36013907496im -0.3946314234413+0.1879641683128im 5.892415453019+9.32210272754im 98.89353516597+50.05933116186im 39.78378513071-75.10692662408im;
  0.9381411884153+0.08627890730181im 1.208990212119+0.08001085583895im 0.4626445513386+0.1377396154157im 0.5640732960706+0.1930940899081im -0.006104621224184-0.001583212382786im 0.1444312379959+0.08462915186578im 0.1855783265566+0.1188869215868im -0.08773359870338-0.01829888136588im -0.1906494204387-0.1061186713289im 0.1850499768677+0.3818468039884im 0.03425237452596+0.1692935172775im -0.1238640130947+0.1308309654496im -0.958674498466-0.08131416841932im -1.263155569038-0.07914229193852im -0.519866874562-0.1391767040657im -0.6456703956818-0.1993603084078im -0.0135444937546-0.002706295267589im -0.1936407491895-0.0971643485954im -0.2514770877442-0.1394903746048im 0.1097079105922+0.0132823575297im 0.2506605651049+0.1192643764593im -0.421959905566+0.2159811865152im -0.1915538784794+0.07589262845729im -0.03692346001024+0.1931536647953im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
  0.5920300133652+0.1536974538374im 0.4002422356344+0.06235837720354im 0.02667803949091+0.03515090072232im 0.0180226767199+0.02842532182349im -0.02246319041221-0.02420129322653im -0.0002395013170902+0.0005012704310879im -0.0002111839914828+0.0004796542092535im 0.0002048234219145-0.0005376183766866im 0.0002715440193815-0.0007123015472893im -0.01860900323779+0.03660127670022im -0.0005398120829413+4.441547162947e-5im -0.000339009344963-0.0004279774589617im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.614560638717+0.06231511748861im 2.233588132259-0.03551164298656im 5.721514823265-1.25053718594im 11.1639928729-5.442680813353im -44.28137488556+235.8403895891im 51.5602807445-257.6634411485im 73.59746555273-328.8381655477im -176.4367379761+517.2601908065im -0.3952110112777+0.2452070587753im 7.154172605757+7.421715880506im 246.9573102639-86.42437749978im -250.418324894-231.2852622526im;
  0.5920300133652+0.1536974538374im 0.4002422356344+0.06235837720354im 0.02667803949091+0.03515090072232im 0.0180226767199+0.02842532182349im -0.02246319041221-0.02420129322653im -0.0002395013170902+0.0005012704310879im -0.0002111839914828+0.0004796542092535im 0.0002048234219145-0.0005376183766866im 0.0002715440193815-0.0007123015472893im -0.01860900323779+0.03660127670022im -0.0005398120829413+4.441547162947e-5im -0.000339009344963-0.0004279774589617im -1.439860888604+0.04895546971635im -2.633881195844-0.0267849056027im -3.549605142749+0.7544206394283im -5.740426825016+1.226152546197im -2.593510132187+0.601519720689im -15.33339461863+11.14935049419im -23.46504003498+15.39962704479im -2.225540205559-2.205598042913im 11.80916045943-8.886372053423im -2.765148609442+2.217781843327im -4.427636193157+17.12277961199im 7.81250833098+12.15551022739im 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
]

python_RHS = [
    -0.944344574850895 - 0.3146824963368253im,
    -0.944344574850895 - 0.3146824963368253im,
    -0.8519297357101365 - 0.2519186732695253im,
    -0.8519297357101365 - 0.2519186732695253im,
    -0.9923959575399124 - 0.3463404890759101im,
    -0.9923959575399124 - 0.3463404890759101im,
    -0.6950361972746393 - 0.137936797384107im,
    -0.6950361972746393 - 0.137936797384107im,
    -0.8302963101855028 - 0.2392205791250337im,
    -0.8302963101855028 - 0.2392205791250337im,
    -1.269163647380495 - 0.5078585922067731im,
    -1.269163647380495 - 0.5078585922067731im,
    -0.7151133669581426 - 0.1523746919083901im,
    -0.7151133669581426 - 0.1523746919083901im,
    -1.0400364307948 - 0.3720763845471957im,
    -1.0400364307948 - 0.3720763845471957im,
    -0.9381728583395414 - 0.3134111306579754im,
    -0.9381728583395414 - 0.3134111306579754im,
    -0.9795195148596036 - 0.3342905690350086im,
    -0.9795195148596036 - 0.3342905690350086im,
    -0.7774185301356979 - 0.2019311420708476im,
    -0.7774185301356979 - 0.2019311420708476im,
    -0.8994694495974045 - 0.2873875057126654im,
    -0.8994694495974045 - 0.2873875057126654im,
    -0.8255059486541612 - 0.2361671827434417im,
    -0.8255059486541612 - 0.2361671827434417im,
    -0.9749221662374481 - 0.3379823757584646im,
    -0.9749221662374481 - 0.3379823757584646im,
    -0.9013443554086535 - 0.2889120897346196im,
    -0.9013443554086535 - 0.2889120897346196im,
    -0.7977441226861117 - 0.2146718747665322im,
    -0.7977441226861117 - 0.2146718747665322im,
    -0.8421489960569488 - 0.2473248672359638im,
    -0.8421489960569488 - 0.2473248672359638im,
    -0.7365528015594691 - 0.170472589456411im,
    -0.7365528015594691 - 0.170472589456411im,
    -0.8071571789710177 - 0.222784401156721im,
    -0.8071571789710177 - 0.222784401156721im,
    -0.7917786327463361 - 0.2110387734598973im,
    -0.7917786327463361 - 0.2110387734598973im,
    -0.7289951826116123 - 0.1615931516693987im,
    -0.7289951826116123 - 0.1615931516693987im,
    -1.103077080839981 - 0.4123524531638587im,
    -1.103077080839981 - 0.4123524531638587im
]

# python_LHS[3:end, :] \ python_RHS[3:end, :] - 
python_LHS[1:end-3, :] \ python_RHS[1:end-3, :]

36×1 Matrix{ComplexF64}:
      1.0326250792270304 - 0.24323379309679716im
    -0.41948441323123875 + 0.2910169927693059im
   -0.008180340369114511 + 0.0029930922524584514im
    0.004944454138192598 - 0.0030520480718408865im
  -0.0013784483375195884 + 0.0009279372443757954im
 -0.00043016049039839216 - 7.539299830767951e-5im
   0.0001774175822055037 + 0.0003216668878575168im
    9.474718833825529e-5 - 0.00025061129468487473im
 -0.00016505047521108543 + 4.1739167470322964e-5im
   -8.971593079862183e-7 - 5.517949981123105e-6im
    8.684729172452095e-5 - 0.0006373486228851942im
    5.122513940158695e-5 + 0.0005080359011962397im
      1.5879638001850023 - 0.2263992250525826im
                         ⋮
      0.6713358697869547 + 1.0862344709235858im
    -0.16420534379570118 - 0.06653774023101763im
   0.0018269107242358785 + 0.0008555084940339952im
  -0.0015850618943016206 - 0.0012202925554561294im
   -9.290016950923468e-7 + 7.202165563928112e-7im
   -8.137529741104232e-7 - 9.087047052045925e

In [379]:
sys.matrix.LHS[3:end,:] \ sys.matrix.RHS[3:end]

36-element Vector{Complex{BigFloat}}:
    -0.182994443465304 + 0.243692963628448im
    -0.999999904451556 - 9.0678940974365e-08im
 -0.000166650314726152 + 0.000985933579859211im
  -8.9650685179566e-06 - 1.74491259614831e-06im
  4.59951125417711e-06 + 1.35638663565675e-06im
  0.000133336893673452 - 0.000460984149021182im
 -4.52058035130178e-06 + 0.000762622273408509im
  -0.00117427343633691 - 1.28790332062515e-05im
  0.000547494572092189 - 0.000373557487718695im
  0.000940686096927434 - 0.000119245270060284im
 -0.000470360007025106 - 0.000183802896656857im
  7.24156574389775e-05 + 0.000320888689494397im
     0.372344534784219 + 0.260527352782049im
                       ⋮
     0.372344246944181 + 0.260527296555068im
    -0.164205284107325 - 0.0665377174938584im
  -0.00210212416076749 - 0.00194634227057167im
 -3.66010170173032e-06 - 5.58923243788607e-06im
   0.00182763757623694 + 0.000858017221382845im
 -0.000691248919363174 - 0.000174736193354286im
  2.63135643547685e-07 + 8.85951317282